<a href="https://colab.research.google.com/github/sebavassou/LLMRAG/blob/main/LLMRAG_INTO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain faiss-cpu sentence-transformers
!pip install langchain-community
!pip install streamlit
!pip install pypdf
!pip install huggingface_hub[hf_xet]


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 54.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 49.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 92.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [2]:
from google.colab import drive
drive.mount('/content/drive')
from google.colab import userdata


Mounted at /content/drive


In [15]:
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
n_documentos = 3

In [11]:
#import os
#os.environ["OPENAI_API_KEY"] = ""

In [8]:
from langchain.document_loaders import PyPDFLoader

# Carregar o PDF using the raw URL
loader = PyPDFLoader("https://raw.githubusercontent.com/sebavassou/chatbot_fila_into/main/dados/Cartilha-Funcionamento-da-Lista-de-Espera-do-Into_web2.pdf")
documents = loader.load()


In [5]:
from langchain.text_splitter import CharacterTextSplitter

splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
docs = splitter.split_documents(documents)

In [6]:
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import FAISS

# Criar embeddings com SentenceTransformer
embeddings = SentenceTransformerEmbeddings(model_name="all-mpnet-base-v2")

# Indexar os documentos com FAISS
vector_store = FAISS.from_documents(docs, embeddings)


<ipython-input-6-448c70939d1d>:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = SentenceTransformerEmbeddings(model_name="all-mpnet-base-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  war

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [12]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)



In [13]:
from langchain.chains import RetrievalQA

retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 3})

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)


In [14]:
query = "Quais cuidados tenho que tomar antes da cirurgia?"
result = qa_chain({"query": query})

print("Resposta:", result["result"])
print("\nFontes:")
for doc in result["source_documents"]:
    print(doc.page_content[:10])  # Exibe os primeiros 10 caracteres da fonte

<ipython-input-14-c1b08a1e10b4>:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"query": query})


Resposta: Antes da cirurgia, é importante que você tome os seguintes cuidados:

1. **Recadastrar informações**: Atualize seu endereço e número de telefone sempre que houver alterações. Isso pode ser feito presencialmente ou via e-mail.

2. **Comunicar mudanças**: Informe ao Instituto sobre cirurgias realizadas em outras instituições ou se você desistir de realizar a cirurgia.

3. **Comparecer às etapas de preparação**: Não falte a nenhuma das etapas de preparação pré-operatória.

4. **Tirar dúvidas**: Converse com o ortopedista responsável para esclarecer todas as suas dúvidas sobre o procedimento.

5. **Consultar profissionais**: Tire dúvidas sobre o processo de movimentação nas listas de espera com os profissionais do Call Center e da Ouvidoria.

Além disso, mantenha seus dados cadastrais sempre atualizados, pois você será contatado por telefone para cada etapa da preparação.

Fontes:
21
20
Inst
15
14
Inst
13
12
Inst


In [ ]:
# Interface Streamlit
import streamlit as st # Import streamlit

st.title("Chatbot INTO - Lista de Espera")

if "history" not in st.session_state:
    st.session_state.history = []

user_input = st.text_input("Digite sua pergunta:")

if st.button("Enviar") and user_input:
    result = qa_chain({"query": user_input})
    resposta = result["result"]
    fontes = [doc.page_content[:100] for doc in result["source_documents"]]
    st.session_state.history.append({"pergunta": user_input, "resposta": resposta, "fontes": fontes})

# Exibe o histórico de perguntas e respostas
for chat in reversed(st.session_state.history):
    st.markdown(f"**Você:** {chat['pergunta']}")
    st.markdown(f"**Chatbot:** {chat['resposta']}")
    with st.expander("Fontes"):
        for fonte in chat["fontes"]:
            st.write(fonte)

2025-04-17 12:44:50.913 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-17 12:44:51.112 
  command:

    streamlit run /usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-04-17 12:44:51.114 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-17 12:44:51.116 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-17 12:44:51.117 Session state does not function when running a script without `streamlit run`
2025-04-17 12:44:51.118 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-17 12:44:51.120 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-17 12:44:51.121 Thread 'MainThread': missing ScriptRunContext! 

In [ ]:
!streamlit run /usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.82.197.113:8501

  Stopping...
^C
